In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import backend as K
import matplotlib.pyplot as plt
import pandas as pd
import datetime

# Load classification model

In [2]:
classification_model = tf.keras.models.load_model('TrainedModel/trainedModel.h5')

2022-03-22 16:01:20.159444: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-22 16:01:20.736153: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22306 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:3b:00.0, compute capability: 8.6


# Load dataset
### And convert it to numpy array

In [3]:
train_X = pd.read_csv('/home/ege/Repo/SideChannel-AdversarialAI/Tensorflow/DataSet/trainX13.csv', header=None)
train_Y = pd.read_csv('/home/ege/Repo/SideChannel-AdversarialAI/Tensorflow/DataSet/trainY13.csv', header=None)

trainY = train_Y.to_numpy()
trainX = train_X.to_numpy()
trainX = np.expand_dims(trainX,axis=2)

# Normalize dataset

In [4]:
minimum = np.amin(trainX)
maximum = np.amax(trainX)

trainX_normalized = (trainX-minimum)/(maximum-minimum)

trainX_tensor =  tf.convert_to_tensor(trainX_normalized)
output = tf.nn.max_pool1d(trainX_tensor, 2, 2, padding='VALID')

2022-03-22 16:01:27.902336: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


# Define Sampling layer as a subclass of keras.layers.Layer
## Sampling layer: Layer that samples a random point in latent space from a distribution with a mean and variance

In [5]:

class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

## Define latent space dimension

In [6]:
latent_dim = 2

# Encoder

In [7]:

encoder_inputs = keras.Input(shape=(3000,1))

x = layers.Conv1D(256,16,strides=2,padding='same',activation='relu')(encoder_inputs)#possibly update kernel_initializer
#x = layers.MaxPooling1D(pool_size = 4,strides = 4, padding = 'same')(x)

x = layers.Conv1D(128,16,strides=2,padding='same',activation='relu')(x)#possibly update kernel_initializer
#x = layers.MaxPooling1D(pool_size = 4,strides = 4, padding = 'same')(x)

x = layers.Conv1D(64,8,strides=3,padding='same',activation='relu')(x)#possibly update kernel_initializer
#x = layers.MaxPooling1D(pool_size = 4,strides = 4, padding = 'same')(x)

x = layers.Conv1D(32,8,strides=2,padding='same',activation='relu')(x)#possibly update kernel_initializer
#x = layers.MaxPooling1D(pool_size = 4,strides = 4, padding = 'same')(x)

shape_before_flattening = K.int_shape(x)

flatten_1 = layers.Flatten()(x)

#x = layers.LSTM(32,activation='tanh',recurrent_activation='hard_sigmoid',use_bias=True,kernel_initializer='VarianceScaling',recurrent_initializer = 'orthogonal',bias_initializer='Zeros', return_sequences = True)(flatten_1) #Variance Scaling


#x = layers.Dense(64 , activation="relu")(flatten_1)
#x = layers.Dense(32 , activation="relu")(x)
x = layers.Dense(16 , activation="relu")(flatten_1)

z_mean = layers.Dense(latent_dim, name="z_mean",kernel_initializer='Zeros',bias_initializer = 'Zeros')(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var",kernel_initializer='Zeros',bias_initializer = 'Zeros')(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 1500, 256)    4352        ['input_1[0][0]']                
                                                                                                  
 conv1d_1 (Conv1D)              (None, 750, 128)     524416      ['conv1d[0][0]']                 
                                                                                                  
 conv1d_2 (Conv1D)              (None, 250, 64)      65600       ['conv1d_1[0][0]']               
                                                                                            

# Decoder

In [8]:

#DECODER
latent_inputs = keras.Input(shape=(latent_dim,))


x = layers.Dense(16 , activation="relu")(latent_inputs)
#x = layers.Dense(32 , activation="relu")(x)
#x = layers.Dense(64 , activation="relu")(x)

#x = layers.LSTM(32,activation='tanh',recurrent_activation='hard_sigmoid',use_bias=True,kernel_initializer='VarianceScaling',recurrent_initializer = 'orthogonal',bias_initializer='Zeros', return_sequences = True)(x) #Variance Scaling


x = layers.Dense(np.prod(shape_before_flattening[1:]), activation="relu")(x)
x = layers.Reshape(shape_before_flattening[1:])(x)

x = layers.Conv1DTranspose(32, 8, activation="relu", strides=2,padding='same')(x)
x = layers.Conv1DTranspose(64, 8, activation="relu", strides=3,padding='same')(x)
x = layers.Conv1DTranspose(128, 16, activation="relu", strides=2,padding='same')(x)
x = layers.Conv1DTranspose(256, 16, activation="relu", strides=2,padding='same')(x)


decoder_outputs = layers.Conv1DTranspose(1, 16, padding="same",activation="sigmoid")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense_1 (Dense)             (None, 16)                48        
                                                                 
 dense_2 (Dense)             (None, 4000)              68000     
                                                                 
 reshape (Reshape)           (None, 125, 32)           0         
                                                                 
 conv1d_transpose (Conv1DTra  (None, 250, 32)          8224      
 nspose)                                                         
                                                                 
 conv1d_transpose_1 (Conv1DT  (None, 750, 64)          16448     
 ranspose)                                                 

# Defining subclass VAE
## VAE is a subclass of keras.Model class

In [9]:

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction),axis=(1)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=0))
            total_loss = reconstruction_loss + kl_loss
            #total_loss = reconstruction_loss #ABSOLUTELY CHANGE!
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

### Cut dataset

In [10]:
pooled_dataset = output.numpy()

In [11]:



A_Class = 0
B_Class = 1
    
trainA = pooled_dataset[A_Class::14]
trainB = pooled_dataset[B_Class::14]

trainA_Out = trainY[A_Class::14]
trainB_Out = trainY[B_Class::14]

finalX = np.append(trainA, trainB,axis=0)
finalY = np.append(trainA_Out,trainB_Out)

#finalX.shape
finalX.shape


(240, 3000, 1)

# Train model

In [12]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
history = vae.fit(finalX, epochs=25, batch_size=32)

Epoch 1/25


2022-03-22 16:01:50.823993: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


8/8 [==============================] - 4s 39ms/step - loss: 1889.9231 - reconstruction_loss: 1655.6774 - kl_loss: 0.0028
Epoch 2/25
8/8 [==============================] - 0s 21ms/step - loss: 1253.2911 - reconstruction_loss: 1201.4658 - kl_loss: 0.0048
Epoch 3/25
8/8 [==============================] - 0s 21ms/step - loss: 1116.2729 - reconstruction_loss: 1110.1599 - kl_loss: 4.5083e-04
Epoch 4/25
8/8 [==============================] - 0s 21ms/step - loss: 1087.3629 - reconstruction_loss: 1076.1454 - kl_loss: 3.3644e-04
Epoch 5/25
8/8 [==============================] - 0s 21ms/step - loss: 1066.7696 - reconstruction_loss: 1073.9966 - kl_loss: 2.7192e-04
Epoch 6/25
8/8 [==============================] - 0s 21ms/step - loss: 1076.0299 - reconstruction_loss: 1071.3778 - kl_loss: 2.4867e-04
Epoch 7/25
8/8 [==============================] - 0s 21ms/step - loss: 1057.5968 - reconstruction_loss: 1059.0328 - kl_loss: 2.3448e-04
Epoch 8/25
8/8 [==============================] - 0s 21ms/step - lo

In [14]:
import os

#z_mean, z_log_var, z = vae.encoder.predict(finalX[0].T)
#print(z)
#reconstructed_x = vae.decoder.predict(z)

#classification_model.evaluate(trainX_normalized,trainY)

#results = classification_model.predict(trainX_normalized,trainY)

#x = np.linspace(-5, 5, num=10)
#y = np.linspace(-5, 5, num=10)



path = '~/Repo/SideChannel-AdversarialAI/Tensorflow/TwoClassModels'

Repo/SideChannel-AdversarialAI/Tensorflow/TwoClassModels
# Check whether the specified path exists or not
isExist = os.path.exists(path)

if not isExist:
  
  # Create a new directory because it does not exist 
  os.makedirs(path)
  print("The new directory is created!")


resultMatrix = np.zeros(shape=(14,14))


for classA in range(14):
    for classB in range(14):
        amtCorrect = 0
        
        if(classA != classB):
            print(str(classA)+", "+str(classB))
            
            trainA = pooled_dataset[classA::14]
            trainB = pooled_dataset[classB::14]

            trainA_Out = trainY[classA::14]
            trainB_Out = trainY[classB::14]

            finalX = np.append(trainA, trainB,axis=0)
            finalY = np.append(trainA_Out,trainB_Out)
            
            vae = VAE(encoder, decoder)
            vae.compile(optimizer=keras.optimizers.Adam())
            print("fitting...")
            history = vae.fit(finalX, epochs=15, batch_size=32,verbose=0)
            
            for x_i in x:
                for y_i in y:
                    z = np.array([[x_i,y_i]])
                    reconstructed_x = vae.decoder.predict(z)
                    prediction = np.argmax(classification_model.predict(reconstructed_x))
                    
                    if(prediction == classA or prediction == classB):
                        amtCorrect = amtCorrect + 1
            print("Correct Amount: "+ str(amtCorrect))
            resultMatrix[classA,classB] = amtCorrect
            print("------------")
                        
                        
                        
                    
        

#np.argmax(results,axis=1)


0, 1
fitting...
Correct Amount: 44
------------
0, 2
fitting...
Correct Amount: 98
------------
0, 3
fitting...
Correct Amount: 99
------------
0, 4
fitting...
Correct Amount: 100
------------
0, 5
fitting...
Correct Amount: 100
------------
0, 6
fitting...
Correct Amount: 93
------------
0, 7
fitting...
Correct Amount: 97
------------
0, 8
fitting...
Correct Amount: 47
------------
0, 9
fitting...
Correct Amount: 40
------------
0, 10
fitting...
Correct Amount: 100
------------
0, 11
fitting...
Correct Amount: 31
------------
0, 12
fitting...
Correct Amount: 100
------------
0, 13
fitting...
Correct Amount: 99
------------
1, 0
fitting...
Correct Amount: 21
------------
1, 2
fitting...
Correct Amount: 0
------------
1, 3
fitting...
Correct Amount: 9
------------
1, 4
fitting...
Correct Amount: 9
------------
1, 5
fitting...
Correct Amount: 99
------------
1, 6
fitting...
Correct Amount: 94
------------
1, 7
fitting...
Correct Amount: 3
------------
1, 8
fitting...
Correct Amount: 59
-

### Print reconstructed sample

In [ ]:

classToCut = 8
trainXCUT = trainX_normalized[classToCut::14]
trainYCUT = trainY[classToCut::14]

z_mean, z_log_var, z = vae.encoder.predict(finalX)
reconstructed_x = vae.decoder.predict(z)

#*(maximum-minimum)+minimum

fig = plt.figure(figsize=(35,5))
#plt.plot(results)

sampleToPredict = 15

plt.plot(reconstructed_x[sampleToPredict],label='Reconstruction')
plt.plot(trainXCUT[sampleToPredict],label='Sample')
#plt.plot(data3[0],label=3)
#plt.plot(data4[0],label=4)
#plt.plot(averageArray[0])

plt.legend()
plt.yticks(np.arange(0, 9, 1))
plt.xticks(np.arange(0, 6000, 500))
plt.grid()
#plt.axhline(linewidth=1, color='r')
plt.xlabel("5 ms")
plt.ylabel("PnP timing")
#figure(figsize=(8, 6), dpi=80)
fig.savefig('vis_test.png',dpi=200)